In [158]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [159]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [160]:
import json
import gensim.downloader as api
import numpy as np
import torch
import string
import transformers
import gensim
from transformers import DistilBertModel, DistilBertTokenizer, logging

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import matplotlib.pyplot as plt

nltk.download('stopwords')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [161]:
with open("drive/MyDrive/ITcup/ranking_train.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

In [162]:
data=data[:1000]

In [163]:
stop_words = stopwords.words('english')
for i in range(len(stop_words)):
  stop_words[i] = "".join([char for char in stop_words[i] if char not in "'"])

porter = PorterStemmer()

In [164]:
nltk.download('punkt')
logging.set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [207]:
scores   = []
for i in range(len(data)):
  for j in range(len(data[i]['comments'])):
    scores.append(data[i]['comments'][j]['score'])

exclamations = []
questions = []
links = []
lengths = []
for i in range(len(data)):
  avg_com_len=0
  for j in range(len(data[i]['comments'])):
    avg_com_len += len(data[i]['comments'][j]['text'])
  avg_com_len/=5
  tmp_link = 0
  tmp_length = 0
  for j in range(len(data[i]['comments'])):
    tmp_length = len(data[i]['comments'][j]['text'])/avg_com_len
    tmp_exclamation = data[i]['comments'][j]['text'].count("!")#/tmp_length
    tmp_question = data[i]['comments'][j]['text'].count("?")#/tmp_length
    if ("http" or "www") in data[i]['comments'][j]['text']:
      tmp_link += 1

    exclamations.append(tmp_exclamation)
    questions.append(tmp_question)
    links.append(tmp_link)
    lengths.append(tmp_length)

ZeroDivisionError: ignored

In [166]:
import pandas as pd
df = pd.DataFrame(list(zip(scores[:5000], exclamations, questions, links, lengths)), columns =['Score', '!', '?', 'Has Links', 'Length / Avg len in post'])

In [167]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Score,!,?,Has Links,Length
Score,1.000000,-0.046498,-0.075514,0.281478,-0.433200
!,-0.046498,1.000000,0.139172,-0.020778,0.118079
?,-0.075514,0.139172,1.000000,-0.019201,0.208652
Has Links,0.281478,-0.020778,-0.019201,1.000000,-0.117209
Length,-0.433200,0.118079,0.208652,-0.117209,1.000000


In [168]:
for datum in data:
  datum['text'] = datum['text'].lower()
  datum['text'] = "".join([char for char in datum['text'] if char not in string.punctuation])
  datum['text'] = word_tokenize(datum['text'])
  datum['text'] = [word for word in datum['text'] if word not in stop_words]
  #datum['text'] = [porter.stem(word) for word in datum['text']]
  #datum['text'] = ' '.join(datum['text'])
  for comment in datum['comments']:
    comment['text'] = comment['text'].lower()
    comment['text'] = "".join([char for char in comment['text'] if char not in string.punctuation])
    comment['text'] = word_tokenize(comment['text'])
    comment['text'] = [word for word in comment['text'] if word not in stop_words]
    #comment['text'] = [porter.stem(word) for word in comment['text']]
    #comment['text'] = ' '.join(comment['text'])

In [169]:
texts = []
comments = []

for i in range(len(data)):
  texts.append(data[i]['text'])
  for j in range(len(data[i]['comments'])):
    comments.append(data[i]['comments'][j]['text'])

In [170]:
from gensim.models import Word2Vec
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")

In [171]:
similarity=[]

sentences=comments[0:5]
target_sentence=texts[0]

for i in range(len(texts)):
  sentences_similarity = np.zeros(len(sentences))
  w2v_vocab = model.vocab.keys()
  sentences=comments[5*i:5+5*i]
  target_sentence=texts[i]
  for idx, sentence in enumerate(sentences):
    sentence_words = [w for w in sentence if w in w2v_vocab]
    if (len(sentence_words) == 0):
      sentence_words=['fortune']
    target_sentence_words = [w for w in target_sentence if w in w2v_vocab]
    if (len(target_sentence_words) == 0):
      target_sentence_words=['fortune']
    sim = model.n_similarity(target_sentence_words, sentence_words)
    sentences_similarity[idx] = sim

  result = list(zip(sentences_similarity))
  for i in range(5):
    similarity+=result[i]

**Корреляция между скором и длиной, а также между скором и наличием ссылок в комментарии более значимые, чем прочие**

In [172]:
df['Similarity'] = similarity
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Score,!,?,Has Links,Length,Similarity
Score,1.000000,-0.046498,-0.075514,0.281478,-0.433200,-0.090561
!,-0.046498,1.000000,0.139172,-0.020778,0.118079,-0.004997
?,-0.075514,0.139172,1.000000,-0.019201,0.208652,0.029139
Has Links,0.281478,-0.020778,-0.019201,1.000000,-0.117209,-0.062589
Length,-0.433200,0.118079,0.208652,-0.117209,1.000000,0.167008
Similarity,-0.090561,-0.004997,0.029139,-0.062589,0.167008,1.000000


In [173]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [174]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(64,32),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(32,16),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(16,5)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [175]:
model = api.load("glove-wiki-gigaword-100")
w2v_vocab = model.vocab.keys()

In [176]:
comms=[]
for i in range(len(comments)):
  if (comments[i] and comments[i][0] in w2v_vocab):
    cur = model.wv.get_vector(comments[i][0])
    for j in range(1, len(comments[i])):
      if (comments[i][j] and comments[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(comments[i][j])], axis = 0)
  comms.append(cur/(len(comments[i])))

<ipython-input-176-b371fe04192b>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector(comments[i][0])
<ipython-input-176-b371fe04192b>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model.wv.get_vector(comments[i][j])], axis = 0)
<ipython-input-176-b371fe04192b>:8: RuntimeWarning: divide by zero encountered in true_divide
  comms.append(cur/(len(comments[i])))


In [177]:
posts=[]
for i in range(len(texts)):
  if (texts[i] and texts[i][0] in w2v_vocab):
    cur = model.wv.get_vector(texts[i][0])
    for j in range(1, len(texts[i])):
      if (texts[i][j] and texts[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(texts[i][j])], axis = 0)
  posts.append(cur/(len(texts[i])))

<ipython-input-177-dbd57cddccf5>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector(texts[i][0])
<ipython-input-177-dbd57cddccf5>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model.wv.get_vector(texts[i][j])], axis = 0)


In [178]:
expected = [
([1,0,0,0,0],
[0,1,0,0,0],
[0,0,1,0,0],
[0,0,0,1,0],
[0,0,0,0,1],)*(int(len(comms)/5)+5)
]
expected[0] = expected[0][:len(comms)]

In [179]:
import torch
import torch.nn as nn
import torch.optim as optim

In [180]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [181]:
!pip install tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [182]:
loss_fn = nn.CrossEntropyLoss()

model = NeuralNetwork()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(5):
  for i in tqdm(range(len(expected[0]))):
    outputs = model.forward(torch.FloatTensor(np.concatenate((comms[i], np.array([similarity[i]] * 28)))))
    expected_flat = torch.FloatTensor(expected[0][i]).flatten()
    loss = loss_fn(outputs, expected_flat)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pass

100%|██████████| 5000/5000 [00:04<00:00, 1103.87it/s]


In [183]:
for i in range(len(comms)):
  outputs = model.forward(torch.FloatTensor(np.concatenate((comms[i], np.array([similarity[i]] * 28)))))
  expected_flat = torch.FloatTensor(expected[0][i]).flatten()


In [184]:
import random
test = []
for j in range(0,5):
  randomlist = []
  for i in range(0,5):
    n = random.randint(0,100)
    randomlist.append(n)
  test.append(randomlist)
test

[[9, 56, 44, 43, 60],
 [71, 13, 83, 18, 88],
 [2, 68, 42, 72, 26],
 [60, 66, 12, 48, 44],
 [51, 53, 79, 49, 17]]

In [185]:
scores_answer = []
for j in range(5):
    max_val = -1
    max_idx = -1
    for i in range(len(test)):
        if test[i][j] > max_val and i not in scores_answer:
            max_val = test[i][j]
            max_idx = i
    scores_answer.append(max_idx)
scores_answer

[1, 2, 4, 3, 0]

**Займёмся Тестовым Датасетом:**

In [198]:
with open("drive/MyDrive/ITcup/ranking_test.jsonl", "r") as f:
    test_data = [json.loads(line) for line in f]

In [202]:
test_data=test_data[:1000]

In [203]:
stop_words = stopwords.words('english')
for i in range(len(stop_words)):
  stop_words[i] = "".join([char for char in stop_words[i] if char not in "'"])

porter = PorterStemmer()

In [205]:
for datum in test_data:
  datum['text'] = datum['text'].lower()
  datum['text'] = "".join([char for char in datum['text'] if char not in string.punctuation])
  datum['text'] = word_tokenize(datum['text'])
  datum['text'] = [word for word in datum['text'] if word not in stop_words]
  for comment in datum['comments']:
    comment['text'] = comment['text'].lower()
    comment['text'] = "".join([char for char in comment['text'] if char not in string.punctuation])
    comment['text'] = word_tokenize(comment['text'])
    comment['text'] = [word for word in comment['text'] if word not in stop_words]

In [ ]:
test_texts = []
test_comments = []

for i in range(len(test_data)):
  texts.append(test_data[i]['text'])
  for j in range(len(test_data[i]['comments'])):
    comments.append(test_data[i]['comments'][j]['text'])

In [206]:
nltk.download('punkt')
logging.set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [196]:
test_scores = []
for i in range(len(test_data)):
  for j in range(len(test_data[i]['comments'])):
    test_scores.append(test_data[i]['comments'][j]['score'])

In [197]:
model = api.load("glove-wiki-gigaword-100")
w2v_vocab = model.vocab.keys()

In [ ]:
test_comms=[]
for i in range(len(test_comments)):
  if (test_comments[i] and test_comments[i][0] in w2v_vocab):
    cur = model.wv.get_vector(test_comments[i][0])
    for j in range(1, len(test_comments[i])):
      if (test_comments[i][j] and test_comments[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(test_comments[i][j])], axis = 0)
  test_comms.append(cur/(len(test_comments[i])))

In [ ]:
test_posts=[]
for i in range(len(test_texts)):
  if (test_texts[i] and test_texts[i][0] in w2v_vocab):
    cur = model.wv.get_vector(test_texts[i][0])
    for j in range(1, len(test_texts[i])):
      if (test_texts[i][j] and test_texts[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(test_texts[i][j])], axis = 0)
  posts.append(cur/(len(test_texts[i])))